### Objective

At a high level, these strategies all involve providing the model clear and specific instructions for what output it should produce. In this lab, you will:

- Define the output format & specify constraints
- Assign a persona or role
- Include examples
- Experiment with parameter values
- Utilize fallback responses
- Add contextual information
- Structure prompts with prefixes or tags
- Use system instructions
- Break down complex tasks
- Demonstrate the chain-of-thought
- Implement prompt iteration strategies to improve your prompts version by version

### Task 1. Initialize Vertex AI in a Colab Enterprise notebook

Upgrade the Vertex AI SDK & Restart the Kernel

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

# note: Select Runtime > Restart Session > select Yes to restart the runtime.

In [ ]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

### Import packages

In [ ]:
PROJECT_ID = "qwiklabs-gcp-03-f2605c9303f1"  
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

### Task 2. Load a generative model

In [ ]:
model = GenerativeModel("gemini-2.0-flash-001")

### Task 3. Define the output format & specify constraints

In [ ]:
# store a transcript from a customer's order at a fast food restaurant.

transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [ ]:
# Run the following prompt that attempts to understand a customer's order from a conversation in JSON format, but in an unspecific way.

response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

In [ ]:
# Now run a version of this prompt with more specific instructions about exactly how you would like your output structured. Notice how the JSON output now reflects the key pieces of information you are interested in to understand the user's order:

response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

### Task 4. Assign a persona or role

In [ ]:
# Try an example within the context of a chat. To start, create a chat session with your Gemini model:
chat = model.start_chat()

In [ ]:
# Ask for a response without a persona specified:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

In [ ]:
# Now run a version of this prompt with a role specified. Notice how this output might be more personal and appealing for some users.
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

### Task 5. Include examples

In [ ]:
# Run the code below in a new cell to see how it interprets the Customer Message and explains its response
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

### Task 6. Experiment with parameter values

In [ ]:
# Run the code from the following cell a few times. The temperature and top_p parameters which lead to variety in responses are set to low values, so the output should be the same, or very close to the same, each time.
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

In [ ]:
# Now run this version of the code a few times. You'll see that the higher temperature and top_p parameter values now lead to more varied results. Some of the results, however, may be a little too random and not make very much sense. If you want variety in your responses, you'll need to experiment with parameters to determine the right balance of creativity and reliability.
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

### Task 7. Utilize fallback responses

In [ ]:
# Copy the following code cells to your notebook and run them to see the model decline to talk about off-topic queries while still answering on-topic queries.
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

In [ ]:
# You'll also want to test that the model does not reject on-topic questions, so make sure to try out a variety of questions that the model should not decline to answer. Here is one:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

### Task 8. Add contextual information

# Imagine you work for a grocery store chain and want to provide users a way of finding items in your store easily.

# Run the following prompt to inquire about the aisle numbers of some items that can be found in a grocery store.

response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

In [ ]:
# Now run a version of this prompt where you provide it that information in what is called the "context" part of the prompt:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)


### Task 9. Structure prompts with prefixes or tags

In [ ]:
# Notice how the XML-style tags (like <OBJECTIVE_AND_PERSONA>) divide up sections of the prompt and other prefixes like Name: identify other key pieces of information. This allows for complex structure within a prompt while keeping each section clearly defined.

prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

### Task 10. Use system instructions

In [ ]:
# You can include prompt components like those you've explored above in each call to a model, or you can pass them to the model upon instantiation as system instructions.

# Paste the code below into a new code cell and run it.

# Notice how the prompt passed to the generate_content() function doesn't mention music at all, but the model still responds based on the persona & instructions passed to it as a system_instruction.

# Note: Content provided as a system instructions are billed as though they were passed in as part of each prompt at generation time.

system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-1.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)


### Task 11. Demonstrate Chain-of-Thought

# Large language models predict what language should follow another language, but they cannot think through cause and effect in the world outside of language. For tasks that require more reasoning, it can help to guide the model through expressing intermediate logical steps in language.

# Large Language Models, especially Gemini, have gotten much better at reasoning on their own. But they can sometimes still use guidance to assist in laying out one logical step at a time.

# Notice in the code cell below that you will pass a generation_config parameter to the generate_content() function. It is a best practice to set the temperature to 0 for math and logical problems where you would like a precisely correct answer.

question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)